<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**train**

In [ ]:
!pip install wandb
import torch,wandb,os,warnings,csv
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [48]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [10, 20, 50,100,200]
        },
        'dropout': {
            'values': [0.3, 0.4, 0.5,0.6,0.7,0.8]
        },
        'batch_size': {
            'values': [16, 32, 25,64]
        },
        'hidden_size': {
            'values': [8,16,24,32, 64, 128,256]
        },
        'Latent_size': {
            'values': [8,12,16,24,32,64]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, entity="aviax1", project="myproj")

Create sweep with ID: nuavldej
Sweep URL: https://app.wandb.ai/aviax1/myproj/sweeps/nuavldej


In [ ]:
class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])

class autoencoder(nn.Module):
    def __init__(self,hidden_size,lv_size):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size), 
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
             nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

def train():
  config_defaults = {'dropout': 0.5,'num_epochs': 10,'batch_size': 100,'learning_rate': 1e-3,'hidden_size': 128,'Latent_size': 48}
  wandb.init(config=config_defaults,project="myproj")
  config = wandb.config
  (xtrain,ytrain), (xtest,ytest) = mnist.load_data()
  num_epochs=config.num_epochs        #
  batch_size=config.batch_size        #
  image_size=784                      #
  hidden_size=config.hidden_size      #
  lv_size =config.Latent_size         # Latent Variable 
  learning_rate=config.learning_rate  #
  cret = nn.MSELoss()                 # criterion
  dropout = config.dropout            #
  warnings.filterwarnings('ignore')   #
  model = autoencoder(hidden_size,lv_size)
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model.to(device)
  by_digit=0
  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=1e-6)
  dataloader = DataLoader(DigitDataSet(xtrain[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=6)
  for epoch in range(num_epochs):
    for data in dataloader:
      imgs = Variable(data.view(data.size(0), -1)).to(device)
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    wandb.log({"loss": loss.data})
wandb.agent(sweep_id, train)